In [0]:
from IPython.display import clear_output

In [0]:
!pip3 install pycodestyle flake8 pycodestyle_magic
clear_output()

In [0]:
%load_ext pycodestyle_magic

# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
!wget http://www.gutenberg.org/files/2600/2600-0.txt -O peace.txt
clear_output()

In [5]:
text = open('peace.txt', 'r').read()[2:]
len(text)

3227579

In [0]:
import re

In [0]:
regex = r"([\!\"\#\$\%\&\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~\“\”\‘\’\—]|\s)+"

def preprocess_text(text):
    return re.sub(regex, " ", text.lower())

text = preprocess_text(text)
assert len(text) == 3141169

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [0]:
from collections import Counter
import nltk
from sklearn.base import TransformerMixin

In [0]:
class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}

    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings
        """
        self.itos = list(set("".join(text)))
        self.stoi = {token: index for index, token in enumerate(self.itos)}
        text = [[self.stoi[char] for char in s] for s in text]

        while len(self.itos) < self.vocab_size:
            new_token = Counter([(s[i], s[i + 1])
                                 for s in text
                                 for i in range(len(s) - 1)]
                                ).most_common(1)[0][0]
            # most common bigram in the text
            new_id = len(self.itos)

            self.itos.append(new_token)
            self.stoi[new_token] = new_id

            # find occurences of the new_token in the text
            # and replace them with new_id
            for i, s in enumerate(text):
                j = 0
                while j < len(s) - 1:
                    if (s[j], s[j + 1]) == new_token:
                        text[i][j] = new_id
                        text[i].pop(j + 1)
                    j += 1

        return self

    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        text = [[self.stoi[char] for char in s] for s in text]
        for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text
            # and replace them with token_id
            if type(token) == tuple:
                for i, s in enumerate(text):
                    j = 0
                    while j < len(s) - 1:
                        if (s[j], s[j + 1]) == token:
                            text[i][j] = token_id
                            text[i].pop(j + 1)
                        j += 1
        return text

    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = "" if tok > len(self.itos) else self.itos[tok]
        if type(result) == tuple:
            return self.decode_token(result[0]) + self.decode_token(result[1])
        return result

    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))

In [0]:
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
import numpy as np


start_token = vocab_size
end_token = vocab_size + 1


class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = np.full((self.vocab_size,
                              self.vocab_size,
                              self.vocab_size),
                             delta)

    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab
        for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = np.array([self.get_proba(a, b, i, tau)
                           for i in range(self.vocab_size)])
        return result

    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        result = (self.proba[a][b][c] ** (1 / tau)) / \
                 ((self.proba[a][b] ** (1 / tau)).sum())
        return result

    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        for s in text:
            s = [start_token] + s + [end_token]
            for i in range(len(s) - 2):
                self.proba[s[i]][s[i + 1]][s[i + 2]] += 1

        return self


lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    beam = [((input_seq[-2], input_seq[-1], token),
             lm.get_proba(input_seq[-2], input_seq[-1], token, tau=tau))
            for token in np.argsort(lm.infer(input_seq[-2],
                                             input_seq[-1],
                                             tau=tau))[::-1][:k]]

    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if snt[-1] == end_token:
                candidates.append(snt)
                candidates_proba.append(snt_proba)
            else:
                proba = lm.infer(snt[-2], snt[-1], tau)
                best_k = np.argsort(proba)[::-1][:k]
                for candidate in best_k:
                    candidates.append(snt + (candidate,))
                    candidates_proba.append(snt_proba * lm.get_proba(snt[-2],
                                                                     snt[-1],
                                                                     candidate,
                                                                     tau=tau))
        beam = [(candidates[i], candidates_proba[i])
                for i in np.argsort(candidates_proba)[::-1][:k]]
    return beam

In [21]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for s, proba in result:
    print(f"{bpe.decode(input1)}{bpe.decode(s[2:])};\tlog proba: {proba}.")

horse with a smill ;	log proba: 0.6841029778083704.
horse was not been sa;	log proba: 0.0417781077348771.
horse was sold not be;	log proba: 0.02009117223303252.
horse when said not ;	log proba: 0.019575605374780952.
horse the countess mary ;	log proba: 0.012419496299872266.
horse who had been sa;	log proba: 0.01086774490413632.
horse with his fack ;	log proba: 0.007780060616618096.
horse the counderstand wi;	log proba: 0.007335131282565679.
horse the cound him and s;	log proba: 0.007187195139002838.
horse the cound him and w;	log proba: 0.0057511169654040715.


In [22]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for s, proba in result:
    print(f"{bpe.decode(input1)}{bpe.decode(s[2:])};\tlog proba: {proba}.")

here with a smill;	log proba: 0.7304666014228688.
here was not been s;	log proba: 0.04874047901633911.
here was sold not b;	log proba: 0.02279761621687215.
here when said no;	log proba: 0.020902320798071278.
here who had been s;	log proba: 0.012678867502004655.
here was she was not b;	log proba: 0.009807273664700423.
here with his heas;	log proba: 0.009110050541107705.
here with his fack;	log proba: 0.008307338307196785.
here with his hear ;	log proba: 0.007996051526025703.
here was not seemp;	log proba: 0.006440570045187267.


In [23]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
for s, proba in result:
    print(f"{bpe.decode(input1)}{bpe.decode(s[2:])};\tlog proba: {proba}.")

what;	log proba: 0.02443609022556391.
whated;	log proba: 0.011341031957494033.
whated to him;	log proba: 0.0002072659731327774.
what theight;	log proba: 5.717776867622187e-05.
whated to himself;	log proba: 1.6978380760763765e-05.
whated to himself it ;	log proba: 1.702154199055581e-06.
whated to himself i ;	log proba: 1.4922512460114212e-06.
whated to himself sa;	log proba: 9.303074810315937e-07.
whated to himself he ha;	log proba: 8.943145534674918e-07.
whated to himself he w;	log proba: 8.367494787615383e-07.


In [24]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for s, proba in result:
    print(f"{bpe.decode(input1)}{bpe.decode(s[2:])};\tlog proba: {proba}.")

gun and with a smill;	log proba: 0.24096440857242407.
gun been said not ;	log proba: 0.16245309491378124.
gun and said not been;	log proba: 0.10901821889589187.
gun and so mussion of ;	log proba: 0.03893031847254916.
gun but was not been;	log proba: 0.03675416320132029.
gun and so mussion hi;	log proba: 0.03249013770009046.
gun and so must been ;	log proba: 0.03156503196883071.
gun said not been ;	log proba: 0.025207662054596385.
gun and said not see;	log proba: 0.021695852287233024.
gun but was sold no;	log proba: 0.02008459268146401.


In [27]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = 2 ** (-(sum([np.log(lm.get_proba(snt[i-2], snt[i-1], snt[i]))
                          for i in range(2, len(snt))])) / (len(snt) - 2))
    return result


perplexity(tokenized_text[0], lm)

6.092798992818707